# 00-Connect database

In [2]:
import sqlite3
import pandas as pd
from pathlib import Path

SQL_OUT = Path("../sql_outputs/takeaway/")
SQL_OUT.mkdir(parents=True, exist_ok=True)

conn = sqlite3.connect("../databases/takeaway.db")

In [3]:
# Checking DB Connection is in place
# pd.read_sql_query("SELECT 1 AS ok", conn)

# 01-Price distribution of menu items

In [4]:
sql = """
SELECT price, COUNT(*) 
FROM menuitems 
GROUP BY price 
ORDER BY price
"""
df_result = pd.read_sql_query(sql, conn)

df_result.to_csv(
    SQL_OUT / "01_price_distribution.csv",
    index=False
)

df_result

,price,COUNT(*)
0,0.00,99
1,0.01,3
2,0.10,14
3,0.15,1
4,0.20,89
...,...,...
1466,144.00,2
1467,145.00,5
1468,147.00,1
1469,149.99,1


# 02-Resto distribution per location

In [5]:
sql = """
SELECT city, COUNT(*) as num_restaurants 
FROM restaurants 
GROUP BY city
"""
df_result = pd.read_sql_query(sql, conn)

df_result.to_csv(
    SQL_OUT / "02_resto_distribution_by_location.csv",
    index=False
)

df_result

,city,num_restaurants
0,,3
1,1040,1
2,Aalst,38
3,Aalter,4
4,Aarschot,8
...,...,...
525,Zulte,3
526,Zutendaal,1
527,Zwevegem,2
528,Zwijnaarde,1


# 03-Top 10 rated pizza

In [6]:
sql = """
SELECT distinct(r.name), r.ratings
from restaurants as r
join categories as c
on r.primarySlug = c.restaurant_id
where c.name like '%Pizza%'
order by r.ratings DESC
limit 10
"""
df_result = pd.read_sql_query(sql, conn)

df_result.to_csv(
    SQL_OUT / "03_top_10_rated_pizza.csv",
    index=False
)

df_result

,name,ratings
0,Kebap - Pizza De Brug,5.0
1,De Echte Eethuis Carlos,5.0
2,Munchies Ieper,5.0
3,Trend Kebab & Pizzeria,5.0
4,La Trinacria Bottega,5.0
5,Snack-It,5.0
6,Pizza Leie,4.9
7,The Black Horse,4.9
8,Pyramide,4.9
9,Baskent Meerhout,4.9


# 04-Map locations offering kapsalons and their average price

In [15]:
sql = """
SELECT 
    r.name AS restaurant_name,
    r.latitude,
    r.longitude,
    AVG(m.price) AS avg_price,
    r.ratings
FROM restaurants AS r
JOIN menuitems AS m
    ON r.primarySlug = m.primarySlug
WHERE LOWER(m.name) LIKE '%kapsalon%'
GROUP BY r.name, r.latitude, r.longitude, r.ratings
ORDER BY avg_price DESC
"""

df_result = pd.read_sql_query(sql, conn)

# Save to CSV
df_result.to_csv(
    SQL_OUT / "04_kapsalon_location_avg_price.csv",
    index=False
)

df_result


,restaurant_name,latitude,longitude,avg_price,ratings
0,Melita Original,50.905063,3.528835,22.785714,4.4
1,Cannur Pizza Pasta Pitta BBQ,51.005845,3.888688,20.844000,4.3
2,Anatolia Hoogstraten,51.396100,4.753590,19.166667,4.1
3,Flamingo Pizza & Kebap,51.355780,4.865092,17.350000,3.6
4,Qubbe Lounge,50.979642,5.491545,16.000000,4.8
...,...,...,...,...,...
862,Foodbar Melisa,50.889930,5.650628,6.000000,3.6
863,Gevara,51.219154,4.419343,6.000000,0.0
864,Snack Babylon Uccle,50.799746,4.336647,6.000000,3.7
865,Street Kap,50.853887,4.353796,5.700000,4.4


# 05-Which restaurants have the best price-to-rating ratio?

In [8]:
sql = """
SELECT 
    r.name,
    r.ratings,
    AVG(m.price) AS avg_price,
    AVG(m.price) / r.ratings AS price_to_rating
FROM restaurants AS r
JOIN menuitems AS m
    ON r.primarySlug = m.primarySlug
WHERE r.ratings > 0
GROUP BY r.name, r.ratings
ORDER BY price_to_rating
"""
df_result = pd.read_sql_query(sql, conn)

df_result.to_csv(
    SQL_OUT / "05_best_price_to_rating_ratio.csv",
    index=False
)

df_result

,name,ratings,avg_price,price_to_rating
0,Le Corner,5.0,2.363333,0.472667
1,El Bocadillos,4.2,2.000000,0.476190
2,Boulangerie Salam,4.1,2.039535,0.497448
3,Pâtisserie Doce Tentação,5.0,2.625000,0.525000
4,Bazi Boulangerie,4.3,2.335294,0.543092
...,...,...,...,...
3473,Mère Liban,4.4,34.547619,7.851732
3474,Wok van Antwerpen,2.5,20.500000,8.200000
3475,Pizza Hut Delivery,2.1,17.344944,8.259497
3476,Japans Restaurant Amatsu,4.4,38.469697,8.743113


# 06-Where are the delivery ‘dead zones’—areas with minimal restaurant coverage?
Just select all restos wtih locations now and use visualization later to identify the delivery dead zones

In [9]:
sql = """
SELECT 
    l.city,
    l.latitude,
    l.longitude,
    COUNT(ltr.restaurant_id) AS restaurant_count
FROM locations AS l
LEFT JOIN locations_to_restaurants AS ltr
    ON l.ID = ltr.location_id
GROUP BY l.city, l.latitude, l.longitude
ORDER BY restaurant_count ASC
"""
df_result = pd.read_sql_query(sql, conn)

df_result.to_csv(
    SQL_OUT / "06_delivery_dead_zones.csv",
    index=False
)

df_result

,city,latitude,longitude,restaurant_count
0,Alveringem,50.938685,2.642273,0
1,Alveringem,50.940326,2.674055,0
2,Alveringem,50.967696,2.704963,0
3,Alveringem,50.975483,2.674576,0
4,Alveringem,50.979436,2.692900,0
...,...,...,...,...
2500,Antwerp,51.215296,4.441809,546
2501,Antwerp,51.197080,4.411548,547
2502,Antwerp,51.216422,4.434121,547
2503,Antwerp,51.220805,4.441732,552


# 07-How does the availability of vegetarian and vegan dishes vary by area?

In [10]:
sql = """
SELECT
    r.name,
    r.latitude,
    r.longitude,
    SUM(c.name LIKE '%Vegetarian%') AS has_vegetarian,
    SUM(c.name LIKE '%Vegan%') AS has_vegan
FROM restaurants AS r
JOIN categories AS c
    ON r.primarySlug = c.restaurant_id
GROUP BY r.name, r.latitude, r.longitude
HAVING SUM(c.name LIKE '%Vegetarian%') + SUM(c.name LIKE '%Vegan%') > 0
"""
df_result = pd.read_sql_query(sql, conn)

df_result.to_csv(
    SQL_OUT / "07_veggie_vegan_by_area.csv",
    index=False
)

df_result

,name,latitude,longitude,has_vegetarian,has_vegan
0,'t Frietwinkelken,51.091212,4.151635,0,17
1,'t Krok(r)antje,50.879183,4.728653,0,2
2,2Bangkok Thai,51.057265,3.724585,4,0
3,Aahaar Restaurant,51.215530,4.417400,1,0
4,Aalst Sushi,50.942041,4.054103,0,22
...,...,...,...,...,...
275,Tota Empanadas,51.220123,4.398512,4,3
276,Tota Empanadas 2,51.220123,4.398512,0,3
277,Veggiezond,51.036113,3.710839,0,1
278,Volt,51.161879,4.990860,3,3


# 08-Identify the World Hummus Order (WHO): top 3 hummus serving restaurants

In [11]:
sql = """
SELECT DISTINCT
    r.name,
    r.ratings
FROM restaurants AS r
JOIN menuitems AS m
    ON r.primarySlug = m.primarySlug
WHERE m.name LIKE '%Hummus%'
ORDER BY r.ratings DESC
LIMIT 3
"""
df_result = pd.read_sql_query(sql, conn)

df_result.to_csv(
    SQL_OUT / "08_WHO_top_3_hummus.csv",
    index=False
)

df_result

,name,ratings
0,Punto Caffè,5
1,Royal Donuts & Bagels,5
2,Restaurant le Royal,5


# 09-Which restaurants in Leuven provide best balance between rating and price?

In [12]:
sql = """
SELECT 
    r.name,
    r.address,
    AVG(m.price) AS avg_price,
    r.ratings,
    AVG(m.price) / r.ratings AS price_to_rating
FROM restaurants AS r
JOIN menuitems AS m
    ON r.primarySlug = m.primarySlug
WHERE r.city = 'Leuven'
GROUP BY r.name, r.ratings
HAVING r.ratings > 0
ORDER BY price_to_rating
LIMIT 20
"""
df_result = pd.read_sql_query(sql, conn)

df_result.to_csv(
    SQL_OUT / "09_best_Leuven_price_rating.csv",
    index=False
)

df_result

,name,address,avg_price,ratings,price_to_rating
0,De Friettoerist,Sint-Jacobsplein 13,2.722018,3.6,0.756116
1,Frietcafé l'Etoile d'Or,Tiensevest 16,3.632197,4.7,0.772808
2,Frituur Den Dreef,Naamsesteenweg 58,3.770714,4.5,0.837937
3,Sahtain,Louis Melsensstraat 5,4.238095,4.8,0.882937
4,Frituur Het Grotteke,Heilige-Geeststraat 102,4.081356,4.5,0.906968
5,Fritboutique,Tiensestraat 58,3.737500,4.1,0.911585
6,De Smullende Heks,Diestsestraat 250,4.176147,4.3,0.971197
7,Falafel Top,Diestsestraat 159,4.887500,4.9,0.997449
8,The Sandwich Factory,Naamsestraat 86,4.827863,4.8,1.005805
9,Bocata,Maria Theresiastraat 12A,3.867683,3.8,1.017811


# 10-Any restaurants with explicit keto or low carb menu items?

In [13]:
sql = """
SELECT DISTINCT
    r.name AS restaurant_name,
    r.city
FROM menuitems AS m
JOIN restaurants AS r
    ON r.primarySlug = m.primarySlug
WHERE m.name LIKE '%keto%'
   OR m.name LIKE '%low carb%'
   OR m.description LIKE '%keto%'
   OR m.description LIKE '%low carb%'
"""
df_result = pd.read_sql_query(sql, conn)

df_result.to_csv(
    SQL_OUT / "10_restaurants_with_low_carb_options.csv",
    index=False
)

df_result

,restaurant_name,city
0,Yokoso,Mechelen
1,Yokoso,Gent
2,M'adam Traiteur,Brugge
3,Kroketo by 't Huiskombuis,Brugge


# 11-TRYOUTS - locations & locations_to_restaurants used for dead zones iso city of restaurants...

In [14]:
sql = """
SELECT 
    l.city,
    l.latitude,
    l.longitude,
    COUNT(ltr.restaurant_id) AS restaurant_count
FROM locations AS l
LEFT JOIN locations_to_restaurants AS ltr
    ON l.ID = ltr.location_id
GROUP BY l.city, l.latitude, l.longitude
ORDER BY restaurant_count ASC
"""
df_result = pd.read_sql_query(sql, conn)

df_result

,city,latitude,longitude,restaurant_count
0,Alveringem,50.938685,2.642273,0
1,Alveringem,50.940326,2.674055,0
2,Alveringem,50.967696,2.704963,0
3,Alveringem,50.975483,2.674576,0
4,Alveringem,50.979436,2.692900,0
...,...,...,...,...
2500,Antwerp,51.215296,4.441809,546
2501,Antwerp,51.197080,4.411548,547
2502,Antwerp,51.216422,4.434121,547
2503,Antwerp,51.220805,4.441732,552
